# Preparando as lições para o treinamento por aprendizado de máquina

Nesse notebook, a partir de parametros estabelecidos, as lições serão preparadas para então ser realizado o treinamento, para então serem realizadas as validações e inferências do modelo para detecção de faixas de pedestre.

In [ ]:
tamanho_janela = 24 # tamanho em pixels
sobreposicao_de_janelas = 16 # sobreposição das janelas em pixels

## Selecionando as quadrículas SCM das licoes

In [7]:
import geopandas as gpd
import pandas as pd

In [4]:
gdf_scm = gpd.read_file('zip://downloads/SIRGAS_SHP_quadriculaortofoto2017.zip!/SIRGAS_SHP_quadriculaortofoto2017/SIRGAS_SHP_quadriculaortofoto2017.shp')

In [28]:
scm_licoes = pd.read_csv('downloads/imagens/SCMs-licoes.csv')

In [41]:
# scm_licoes.loc[scm_licoes["presente"] == 1]
licoes_utilizadas = scm_licoes.loc[scm_licoes["presente"] == 1, 'scm']

In [49]:
gdf_scm[gdf_scm['qo2010_cod'].isin(licoes_utilizadas)]

,qo2010_cod,geometry
2479,3314-311,"POLYGON ((327777.044 7392618.571, 327770.250 7..."
2480,3313-432,"POLYGON ((327245.386 7392612.299, 327238.571 7..."
2484,3313-412,"POLYGON ((325118.743 7392587.016, 325111.845 7..."
2561,3313-414,"POLYGON ((325125.643 7392010.236, 325118.743 7..."
2793,3316-221,"POLYGON ((332056.790 7390360.987, 332050.159 7..."
2807,3315-211,"POLYGON ((324614.772 7390273.522, 324607.847 7..."
2808,3315-132,"POLYGON ((324083.192 7390267.129, 324076.246 7..."
3035,3325-164,"POLYGON ((336860.179 7388684.936, 336853.733 7..."
3117,3325-332,"POLYGON ((336866.626 7388108.176, 336860.179 7..."
3179,3326-424,"POLYGON ((345376.197 7387624.020, 345370.085 7..."


## Gerando as Labels de faixas de pedestre

## Gerando mascaras de quadras e copas de árvores

Considera lição se a condiçao for:

* mais de 75% dos pixels dentro da lição
* mais de 25% dos pixels estiver fora da quadra viária
* 100% dos pixels dentro do SCM

## Preparando as imagens

### Considerar RGB e NIR

* reescalonar

### Gerando HOG e Local Binary Pattern (LBPH)

### DAta Augmentation das lições das faixas de pedestre

* Rotate 0, 90, 180, 270 
* Cada rotate Flip H e V
* 16 possibilidades de Augmentation